In [ ]:
import subprocess
import fnmatch
from pathlib import Path

import geowombat as gw
from geowombat.util import GeoDownloads
from geowombat.radiometry import BRDF, RadTransforms, landsat_pixel_angles
from geowombat.data import rgbn

import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da

## Download data

In [ ]:
gdl = GeoDownloads()

## Landsat

In [ ]:
gdl.list_gcp('l8', '225/083/*225083_2019*_T*')

In [ ]:
list(gdl.search_dict.keys())

In [ ]:
# gdl.search_dict['LC08/01/225/083/LC08_L1TP_225083_20190131_20190206_01_T1']

In [ ]:
search_wildcards = ['ANG.txt', 'MTL.txt', 'B2.TIF', 'B3.TIF', 'B4.TIF', 'B5.TIF']
outdir = '/media/jcgr/data/imagery/google/p225r083'

In [ ]:
file_info = gdl.download_gcp('l8',
                             outdir=outdir, 
                             search_wildcards=search_wildcards,
                             verbose=1)

In [ ]:
# file_info

## DN -> SR -> BRDF

In [ ]:
rt = RadTransforms()
br = BRDF()

In [ ]:
outdir_angles = '/media/jcgr/data/imagery/google/p225r083/angles'
outdir_brdf = '/media/jcgr/data/imagery/google/p225r083/brdf'

In [ ]:
angle_infos = dict()

kwargs = dict(readxsize=1024, 
              readysize=1024,
              verbose=1,
              separate=True,
              n_workers=1, 
              n_threads=4,
              n_chunks=100,
              overwrite=False,
              compress='lzw')

In [ ]:
for finfo_key, finfo_dict in file_info.items():
    
    meta = rt.get_landsat_coefficients(finfo_dict['meta'].name)
    
    angle_info = landsat_pixel_angles(finfo_dict['angle'].name, 
                                      finfo_dict['B4'].name, 
                                      outdir_angles, 
                                      meta.sensor, 
                                      l57_angles_path='/home/jcgr/Documents/scripts/Python/git_repos/geowombat/bin/ESPA/landsat_angles', 
                                      l8_angles_path='/home/jcgr/Documents/scripts/Python/git_repos/geowombat/bin/ESPA/l8_angles',
                                      verbose=1)
    
    out_brdf = Path(outdir_brdf).joinpath(Path(finfo_key).name + '.tif').as_posix()
    
    with gw.open(angle_info.sza) as sza, gw.open(angle_info.vza) as vza, gw.open(angle_info.saa) as saa, gw.open(angle_info.vaa) as vaa:

        with gw.open([finfo_dict['B2'].name, 
                      finfo_dict['B3'].name, 
                      finfo_dict['B4'].name, 
                      finfo_dict['B5'].name], 
                     band_names=['blue', 'green', 'red', 'nir'],
                     stack_dim='band',
                     chunks=512) as dn:

            # Convert DN to surface reflectance
            dn_sr = rt.dn_to_sr(dn, 
                                sza, saa, vza, vaa, 
                                sensor=meta.sensor, 
                                meta=meta)

            # BRDF normalization
            dn_sr_brdf = br.norm_brdf(dn_sr, 
                                      sza, saa, vza, vaa, 
                                      sensor=meta.sensor, 
                                      wavelengths=dn.band.values.tolist(), 
                                      nodata=0)

            dn_sr_brdf.gw.to_raster(out_brdf, **kwargs)    
    
    angle_infos[finfo_key] = angle_info

## Get coefficients from metadata

In [ ]:
rt = RadTransforms()

# meta = rt.get_coefficients(file_info[1].name)
meta = rt.get_landsat_coefficients('/media/jcgr/data/imagery/temp/google/LC08_L1TP_042034_20160121_20170224_01_T1_MTL.txt')

In [ ]:
meta

## Check data

In [ ]:
# with gw.open([file_info[2].name, file_info[3].name], 
#              band_names=['red', 'nir'],
#              stack_dim='band') as ds:
#     print(ds)
# #     ds.sel(band='nir').plot.imshow()

## Generate pixel angle files

In [ ]:
# angle_info = gw.gen_pixel_angles(file_info[0].name, 
#                                  file_info[2].name, 
#                                  '/media/jcgr/data/imagery/temp/google/angles', 
#                                  meta.sensor, 
#                                  l57_angles_path='/home/jcgr/Documents/scripts/Python/git_repos/geowombat/bin/ESPA/landsat_angles', 
#                                  l8_angles_path='/home/jcgr/Documents/scripts/Python/git_repos/geowombat/bin/ESPA/l8_angles',
#                                  verbose=1)

In [ ]:
# angle_info

In [ ]:
# with gw.open(angle_info.senze) as ds:
#     ds.sel(band=1).plot.imshow(robust=True)

In [ ]:
with gw.open(angle_info.senaz) as ds:
    ds.sel(band=1).plot.imshow(robust=True)

In [ ]:
with gw.open(angle_info.solaz) as solaz:
    with gw.open(angle_info.senaz) as senaz:
        rel_azimuth_angle = xr.ufuncs.fabs(solaz - senaz - 180.0)
        rel_azimuth_angle.sel(band=1).plot.imshow(robust=True)

## Convert to surface reflectance

In [ ]:
angle_info

In [ ]:
# with gw.open(angle_info.sze) as sze:
#     with gw.open(angle_info.vze) as vze:
#         with gw.open(angle_info.saa) as saa:
#             with gw.open(angle_info.vaa) as vaa:
#                 with gw.open([file_info[2].name, file_info[3].name], 
#                              band_names=['red', 'nir'],
#                              stack_dim='band') as dn:

kwargs = dict(readxsize=1024, 
              readysize=1024, 
              n_workers=1, 
              n_threads=4,
              n_chunks=100,
              overwrite=True)

with gw.open('/media/jcgr/data/imagery/temp/google/angles/LC08_L1TP_042034_20160121_20170224_01_T1_solar_zenith.tif', chunks=512) as sze:
    with gw.open('/media/jcgr/data/imagery/temp/google/angles/LC08_L1TP_042034_20160121_20170224_01_T1_sensor_zenith.tif', chunks=512) as vze:
        with gw.open('/media/jcgr/data/imagery/temp/google/angles/LC08_L1TP_042034_20160121_20170224_01_T1_solar_azimuth.tif', chunks=512) as saa:
            with gw.open('/media/jcgr/data/imagery/temp/google/angles/LC08_L1TP_042034_20160121_20170224_01_T1_sensor_azimuth.tif', chunks=512) as vaa:                
                
                with gw.open(['/media/jcgr/data/imagery/temp/google/LC08_L1TP_042034_20160121_20170224_01_T1_B4.TIF', 
                              '/media/jcgr/data/imagery/temp/google/LC08_L1TP_042034_20160121_20170224_01_T1_B5.TIF',
                              '/media/jcgr/data/imagery/temp/google/LC08_L1TP_042034_20160121_20170224_01_T1_B6.TIF'], 
                             band_names=['red', 'nir', 'swir1'],
                             stack_dim='band',
                             chunks=512) as dn:
                    
#                     dn.gw.to_raster('/media/jcgr/data/imagery/temp/google/dn.tif', **kwargs)

                    # Convert DN to surface reflectance
                    dn_sr = rt.dn_to_sr(dn, sze, saa, vze, vaa, sensor=meta.sensor, meta=meta)
                    
#                     dn_sr.gw.to_raster('/media/jcgr/data/imagery/temp/google/dn_sr.tif', **kwargs)
                    
                    # BRDF normalization
                    dn_sr_brdf = BRDF().norm_brdf(dn_sr, sze, saa, vze, vaa, sensor=meta.sensor, wavelengths=dn.band.values.tolist(), nodata=0)
                    
#                     dn_sr_brdf.gw.to_raster('/media/jcgr/data/imagery/temp/google/dn_sr_brdf.tif', **kwargs)

In [ ]:
dn_sr

In [ ]:
dn_sr_brdf

In [ ]:
dn.sel(band='nir').plot.imshow(robust=True)

In [ ]:
dn_sr.sel(band='nir').plot.imshow(robust=True)

In [ ]:
dn_sr_brdf.sel(band='nir').plot.imshow(robust=True)